In [119]:
from bs4 import BeautifulSoup
import requests
import time
import random
import re
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys
import pandas as pd 
from medscraper import medsearch

In [120]:
options=webdriver.ChromeOptions()
options.add_argument('--incognito')
options.add_argument('--disable-blink-features=AutomationControlled')
options.add_argument('--disable-cache')
options.add_argument('--disable-cookies')

In [121]:
url="https://www.medicinpriser.dk/default.aspx?lng=2"

In [122]:
medicamento="sorafenib"

In [123]:
def tabla_dinamarca():
    table_element = driver.find_element(By.XPATH, "//table[@cellpadding='0' and @border='0' and @cellspacing='0' and @style='width:100%']")
    table_contents = table_element.get_attribute('outerHTML')
    return table_contents

def search_medicine(id_element, medicine_name):
    try:
        driver.find_element(By.ID, id_element).clear()
        driver.find_element(By.ID, id_element).send_keys(medicine_name)
        driver.find_element(By.ID, "ctl00_ctl07_simpleForm_SearchButton").click()
        return tabla_dinamarca()
    except NoSuchElementException:
        return []

driver = webdriver.Chrome(options=options)
driver.get(url)

# First, try to find by 'VirksomtstofBox'
tabla = search_medicine("ctl00_ctl07_simpleForm_VirksomtstofBox", medicamento)

# If the first search fails (returns empty list), try to find by 'LaegemiddelBox'
if not tabla:
    driver.find_element(By.ID, "ctl00_ctl07_simpleForm_VirksomtstofBox").clear()
    tabla = search_medicine("ctl00_ctl07_simpleForm_LaegemiddelBox", medicamento)
#Driver quit
driver.quit()



In [124]:
# driver=webdriver.Chrome(options=options)
# driver.get(url)
# try:
#     driver.find_element(By.ID, "ctl00_ctl07_simpleForm_VirksomtstofBox").send_keys("dasdsa")
#     driver.find_element(By.ID, "ctl00_ctl07_simpleForm_SearchButton").send_keys(Keys.ENTER)
#     try: 
#         tabla=tabla_dinamarca()
#     except NoSuchElementException:
#         try:
#             #Erase the previous search
#             driver.find_element(By.ID, "ctl00_ctl07_simpleForm_VirksomtstofBox").clear()
#             driver.find_element(By.ID, "ctl00_ctl07_simpleForm_LaegemiddelBox").send_keys("dasdsa")
#             driver.find_element(By.ID, "ctl00_ctl07_simpleForm_SearchButton").send_keys(Keys.ENTER)
#             tabla=tabla_dinamarca()
#         except NoSuchElementException:
#             tabla=[]
# except NoSuchElementException:
#     tabla=[]
# finally:
#     driver.quit()

In [125]:
presentaciones = {
        'tableta': ['tab', 'tableta', 'comp', "com", 'comprimidos', 'cpr', "stk"],
        'capsula': ['capsula', 'cap', 'capsulas'],
        'jarabe': ['jarabe', 'jar'],
        'solucion': ['solucion', 'sol', 'sln'],
        'crema': ['crema', 'crm'],
        'suspension': ['suspension', 'susp', 'sus'],
        'inyeccion': ['inyeccion', 'iny', 'inyec']
    }

In [182]:
if tabla == []:
    df = pd.DataFrame()
else:
    soup = BeautifulSoup(tabla, 'html.parser')
    tableprod = soup.find('table', attrs={'id': 'ctl00_ctl07_Results_ResultGridStatic'})
    tableprecio=soup.find('table', attrs={'id': 'ctl00_ctl07_Results_ResultGridScroll'})
    rowsprod = tableprod.find_all('tr') 
    rowsprecio=tableprecio.find_all('tr')
    caract=[]
    for row in rowsprod:
        columns = row.find_all('td')
        for column in columns:
            caract.append(column.text)
    producto=caract[::3]
    producto=[re.sub(r'\d+', '', i) for i in producto]
    presentacion=caract[1::3]
    #paquete=caract[2::3]
    
    caract2=[]
    for row in rowsprecio:
        columns = row.find_all('td')
        for column in columns:
            caract2.append(column.text)
    precio=caract2[4::5]
    precio=[re.sub(r',', '.', i) for i in precio]

    titulos=[]
    for row in rowsprod:
        divs=row.find_all('div')
        for div in divs:
            titulos.append(div.get("title"))
    paquete=titulos[2::3]

    def euro():
        exchangerates='https://openexchangerates.org/api/latest.json?app_id=52adcacdba9c48ce8430667723c02968&base=USD&symbols=MXN,EUR'
        r=requests.get(exchangerates).json()
        euro=r['rates']['MXN']/r['rates']['EUR']
        return euro

    df=pd.DataFrame({'producto':producto, 'presenta':presentacion, 'paquete':paquete, 'precio':precio})
    df[['presentacion','cantidad']]=df.presenta.str.split(" ",expand=True)
    df=df[~df.precio.str.contains("Discont")]
    df["precio"]=df["precio"].astype(float)
    df["precio"]=round(df["precio"]*euro(),2)
df

,producto,presenta,paquete,precio,presentacion,cantidad
0,Nexavar,200 mg,112 stk. (blister)\nfilmovertrukne tabl.,66283.96,200,mg
3,"Sorafenib ""Mylan""",200 mg,112 stk. (blister)\nfilmovertrukne tabl.,62756.20,200,mg


In [181]:
paquete

['200 mg', '200 mg', '200 mg', '200 mg']

In [166]:
#Encontrar todos los div en rowsprod
titulos=[]
for row in rowsprod:
    divs=row.find_all('div')
    for div in divs:
        titulos.append(div.get("title"))

In [167]:
titulos

['Nexavar\n163494',
 '200 mg',
 '112 stk. (blister)\nfilmovertrukne tabl.',
 'Nexavar\n518462',
 '200 mg',
 '112 stk. (blister) (Abacus)\nfilmovertrukne tabl.',
 'Nexavar\n110475',
 '200 mg',
 '112 stk. (blister) (Paranova)\nfilmovertrukne tabl.',
 'Sorafenib "Mylan"\n572301',
 '200 mg',
 '112 stk. (blister)\nfilmovertrukne tabl.']

In [178]:
#Dejar cada 3 elementos a partir del 3

titulos[2::3]

['112 stk. (blister)\nfilmovertrukne tabl.',
 '112 stk. (blister) (Abacus)\nfilmovertrukne tabl.',
 '112 stk. (blister) (Paranova)\nfilmovertrukne tabl.',
 '112 stk. (blister)\nfilmovertrukne tabl.']

In [156]:
#Obtener el texto de los titles en los divs
texto=[]
for div in divs:
    texto.append(div.get('title'))
texto


['Sorafenib "Mylan"\n572301',
 '200 mg',
 '112 stk. (blister)\nfilmovertrukne tabl.']

In [127]:
#Extraer primer número de la variable "paquete"
df['num_prod'] = df['paquete'].str.extract('(\d+)')
df

,producto,presenta,paquete,precio,presentacion,cantidad,num_prod
0,Nexavar,200 mg,112 stk. (blister)filmovertrukne ta ...,66283.96,200,mg,112
3,"Sorafenib ""Mylan""",200 mg,112 stk. (blister)filmovertrukne ta ...,62756.20,200,mg,112


In [128]:
import numpy as np
df['x'] = np.nan  # initialize 'x' column with NaN

for key, values in presentaciones.items():
    for value in values:
        mask = df['paquete'].str.contains(value, case=False, na=False)
        df.loc[mask, 'x'] = key
df

,producto,presenta,paquete,precio,presentacion,cantidad,num_prod,x
0,Nexavar,200 mg,112 stk. (blister)filmovertrukne ta ...,66283.96,200,mg,112,tableta
3,"Sorafenib ""Mylan""",200 mg,112 stk. (blister)filmovertrukne ta ...,62756.20,200,mg,112,tableta


In [136]:
#Show all the text 
df["paquete"][0].split(" ")[3]

'ta'

In [137]:
tabla

'<table cellpadding="0" border="0" cellspacing="0" style="width:100%">\n        <tbody><tr>\n            <td>\n                <div style="padding-top:5px;padding-bottom:5px">\n                    <div id="ctl00_ctl07_Results_LeftGridDiv" style="float:left;overflow:hidden;" class="width282">\n                        <div>\n\t<table class="grid" cellspacing="0" border="0" id="ctl00_ctl07_Results_ResultGridStatic" style="border-collapse:collapse;border:1px solid #505050;border-width:1px 0px 1px 1px;">\n\t\t<tbody><tr style="height:45px;">\n\t\t\t<th scope="col"><a href="javascript:__doPostBack(\'ctl00$ctl07$Results\',\'Sort_20\')">Medicinal product/<br>product number<br>  </a><img src="images/ikon_spm.gif" alt="" id="fbi1" onclick="ShowFactBox(this.id, \'fbb415\')" class="factBoxButton" align="absmiddle"><img src="Images/ikon_sort_ned.gif" alt="Sortér" onclick="__doPostBack(\'ctl00$ctl07$Results\',\'Sort_20\')"></th><th scope="col"><a href="javascript:__doPostBack(\'ctl00$ctl07$Results\'